## Imports

In [ ]:
import influxdb_client
import numpy
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from getpass import getpass
import sys, os
import datetime
import time as time_module
import calendar
import json
import dateutil, pytz
import pandas as pd
import argparse

## Set connection

In [ ]:
### remote (idiotdb)
bucket_remote = ""
org_remote = ""
token_remote = ""
# Store the URL of your InfluxDB instance
url_remote=""

## Check buckets

In [ ]:
### set client to get access to influx
clientV2_remote = influxdb_client.InfluxDBClient(
   url=url_remote,
   token=token_remote,
   org=org_remote
)
### set bucket api
buckets_api_remote = clientV2_remote.buckets_api()

In [ ]:
### warning: times out outside Glasgow (use VPN)
### list buckets (by name)
try:
    #print([x.name for x in buckets_api_remote.find_buckets().buckets])
    database_list=[x.name for x in clientV2_remote.buckets_api().find_buckets().buckets]
    print(database_list)
except:
    print("cannot get buckets")

In [ ]:
### find bucket
buckets_api_remote.find_bucket_by_name('GLADD')

## Query Data

In [ ]:
### set query api
query_api_remote = clientV2_remote.query_api()

In [ ]:
### build query (example for GLADD influx)
# set time period
start_time_str="2023-08-01_23:00:00"
start_time=datetime.datetime.strptime(str(start_time_str), '%Y-%m-%d_%H:%M:%S')
start_time.strftime('%Y-%m-%dT%H:%M:%SZ')
offset_time=60 #s
end_time = start_time + datetime.timedelta(seconds=offset_time)
end_time.strftime('%Y-%m-%dT%H:%M:%SZ')
# set filters
filters={'_measurement':"data"}
# build query str
query_str = ' from(bucket: \"'+bucket_remote+'\") |> range(start: '+start_time.strftime('%Y-%m-%dT%H:%M:%SZ')+', stop: '+end_time.strftime('%Y-%m-%dT%H:%M:%SZ')+')'
for k,v in filters.items():
    query_str+=' |> filter(fn: (r) => r["'+k+'"] == "'+v+'")'
query_str+=' |> yield(name: "mean")'
print(query_str)

In [ ]:
### send query and show result (dataframe version)
query_result_remote = query_api_remote.query_data_frame(org=org_remote, query=query_str)
display(query_result_remote)

In [ ]:
### get timestamp info for 1st element
tmsp=query_result_remote.to_dict()['_time'][0]
print(tmsp.to_pydatetime().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
### get temperature information
results = []
field="temperature"
for index, row in query_result_remote.iterrows():
    if row['_field']==field:
        results.append(row.to_dict())
print(results)